Задача 1: сравнение предложений

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

    кошки (животные)
    UNIX-утилита cat для вывода содержимого файлов
    версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.
sentences.txt

Выполните следующие шаги:

    Скачайте файл с предложениями (sentences.txt).
    Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
    Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
    Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
    Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
    Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
    Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
    Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

Открываем файл sentences.txt

In [1]:
f = open('sentences.txt')

Создаем массив для строк и читаем все строки в массив

In [2]:
sen = []
for line in f:
    sen.append(line.lower().rstrip())
f.close()

Разбиваем строки на слова без пустых слов и создаем словарь.

In [3]:
import re

worlds = {}
sen_worlds = []

index = 0
for line in sen:
    lw = []
    for w in re.split('[^a-z]', line):
        if w:
            lw.append(w)
            if not w in worlds:
                worlds[w] = index
                index = index + 1
    sen_worlds.append(lw)

Делаем массив количества употреблений слова в в каждом предложении.

In [4]:
import numpy as np
matrix = np.zeros((len(sen_worlds), len(worlds)))

for sen_index, s in enumerate(sen_worlds):
    for w in s:
        w_index = worlds[w]
        matrix[sen_index, w_index] = matrix[sen_index, w_index] + 1

Для каждого предложения определяем косинусное расстояние между от первого предложения.

In [5]:
from scipy import spatial

model = matrix[0]
distance = []
result = [(-1, 1), (-1, 1)]

for i in range(0, matrix.shape[0]):
    d = spatial.distance.cosine(matrix[i], model)
    distance.append(d)
    if i > 0 and result[0][1] > d:
        result[1] = result[0]
        result[0] = (i, d)

f = open("submit_1.txt", "w")
f.write(str(result[0][0]) + " " + str(result[1][0]))
f.close()

Выводим результат

In [6]:
!more submit_1.txt

6 4


In [7]:
print(result)

[(6, 0.7327387580875756), (4, 0.77708871496985887)]
